In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("Flight_data.csv")

In [3]:
df.dropna()

,Departure City,Arrival City,Departure Date,Flight Duration,Delay Minutes,Customer ID,Name,Booking Class,Frequent Flyer Status,Route,Ticket Price,Competitor Price,Demand,Origin,Destination,Profitability,Loyalty Points,Churned
0,Wilsonstad,Lake Johnmouth,2023-05-02 20:11:09,1.275810,120,3769,Daniel Oliver,Business,Gold,MEL-BNE,370.638128,382.947396,-0.932755,MEL,LHR,0.632226,4245,True
1,New Brent,Port Wanda,2023-04-21 00:10:14,1.275810,35,3529,Deborah Hall,Economy,Platinum,BNE-SYD,114.529016,394.583641,-1.005569,MEL,SIN,1.265026,833,True
2,South Samanthaberg,Lake Meganside,2023-05-12 15:16:31,0.721110,67,1303,Mary York,Economy,Platinum,MEL-BNE,164.468018,479.832444,1.761384,MEL,LAX,1.141651,2568,True
3,Lake Gracefurt,Jamesberg,2023-06-13 20:53:09,-0.942990,72,2965,Christina Sanchez,Economy,Gold,BNE-SYD,318.903167,286.301632,-0.520139,MEL,LAX,1.129291,284,True
4,Owenborough,Kelleymouth,2023-05-15 23:06:14,-0.388290,101,8779,Dustin Owens,Economy,Silver,BNE-SYD,389.971051,407.463316,-0.665768,BNE,SIN,1.218239,2805,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Brittneychester,South Patricia,2023-06-07 01:09:54,-1.497691,15,6848,Brittney Swanson,Economy,Silver,SYD-MEL,210.236193,275.963390,0.208007,BNE,LAX,1.127112,3522,True
96,South Staceyhaven,West Kenneth,2023-05-19 23:13:31,-1.497691,84,3724,Jessica Johnson,Business,Gold,BNE-SYD,184.330189,246.368703,-0.520139,BNE,SIN,0.569017,3244,True
97,Arnoldtown,Morriston,2023-05-22 11:51:34,0.166410,8,8395,Amber Mcclure,First,Gold,SYD-MEL,464.282801,199.549449,-0.981298,SYD,LAX,0.531339,1484,True
98,Murrayhaven,Lake Jennifermouth,2023-05-27 21:15:35,-0.942990,7,4142,Lisa Luna,First,Gold,BNE-SYD,457.453579,280.391721,0.741980,SYD,LHR,1.483030,1328,True


In [4]:
df.head()

,Departure City,Arrival City,Departure Date,Flight Duration,Delay Minutes,Customer ID,Name,Booking Class,Frequent Flyer Status,Route,Ticket Price,Competitor Price,Demand,Origin,Destination,Profitability,Loyalty Points,Churned
0,Wilsonstad,Lake Johnmouth,2023-05-02 20:11:09,1.27581,120,3769,Daniel Oliver,Business,Gold,MEL-BNE,370.638128,382.947396,-0.932755,MEL,LHR,0.632226,4245,True
1,New Brent,Port Wanda,2023-04-21 00:10:14,1.27581,35,3529,Deborah Hall,Economy,Platinum,BNE-SYD,114.529016,394.583641,-1.005569,MEL,SIN,1.265026,833,True
2,South Samanthaberg,Lake Meganside,2023-05-12 15:16:31,0.72111,67,1303,Mary York,Economy,Platinum,MEL-BNE,164.468018,479.832444,1.761384,MEL,LAX,1.141651,2568,True
3,Lake Gracefurt,Jamesberg,2023-06-13 20:53:09,-0.94299,72,2965,Christina Sanchez,Economy,Gold,BNE-SYD,318.903167,286.301632,-0.520139,MEL,LAX,1.129291,284,True
4,Owenborough,Kelleymouth,2023-05-15 23:06:14,-0.38829,101,8779,Dustin Owens,Economy,Silver,BNE-SYD,389.971051,407.463316,-0.665768,BNE,SIN,1.218239,2805,True


In [8]:
import pandas as pd
from geopy.distance import geodesic

# Step 1: Clean Date Fields
df['Departure Date'] = pd.to_datetime(df['Departure Date'], dayfirst=True, errors='coerce')

# Step 2: Calculate Distance Using Airport Coordinates
# Load airport data with lat/lon (IATA codes) from OpenFlights dataset
airport_cols = ['AirportID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 'Tz database time zone', 'Type', 'Source']
airports = pd.read_csv('airports.dat', names=airport_cols, header=None)

# Filter for airports with valid IATA codes
airports = airports[airports['IATA'].apply(lambda x: isinstance(x, str) and len(x) == 3)]

# Create dict for quick lookup
airport_coords = airports.set_index('IATA')[['Latitude', 'Longitude']].to_dict('index')

def get_distance(row):
    origin = row['Origin']
    dest = row['Destination']
    try:
        coord1 = (airport_coords[origin]['Latitude'], airport_coords[origin]['Longitude'])
        coord2 = (airport_coords[dest]['Latitude'], airport_coords[dest]['Longitude'])
        return geodesic(coord1, coord2).miles
    except KeyError:
        return None  # missing airport code

df['Distance_Miles'] = df.apply(get_distance, axis=1)

# Step 3: Estimate Seats based on Booking Class (example assumptions)
def estimate_seats(row):
    if row['Booking Class'].lower() == 'economy':
        return 180
    elif row['Booking Class'].lower() == 'business':
        return 50
    elif row['Booking Class'].lower() == 'first':
        return 20
    else:
        return 100  # default guess

df['Estimated_Seats'] = df.apply(estimate_seats, axis=1)

# Step 4: Calculate KPIs

# Revenue = Ticket Price (assuming per passenger)
df['Revenue'] = df['Ticket Price']

# Available Seat Miles (ASM) = Estimated Seats * Distance
df['ASM'] = df['Estimated_Seats'] * df['Distance_Miles']

# Revenue Passenger Miles (RPM) = 1 passenger assumed per row * Distance (you may want to adjust if you have passenger counts)
# Since your dataset is per booking (1 customer), RPM = Distance for that booking
df['RPM'] = df['Distance_Miles']  # per passenger

# Load Factor = RPM / ASM
df['Load_Factor'] = df['RPM'] / df['ASM']

# Step 5: Segmenting is just grouping/filtering in analysis, but let's create some example segments:
# You can group by these columns later in Power BI or pandas

# Example: Churned customers count per Booking Class
churn_counts = df.groupby('Booking Class')['Churned'].sum()

# Example: Avg Profitability by Frequent Flyer Status
avg_profit_ff = df.groupby('Frequent Flyer Status')['Profitability'].mean()

print("Churn counts by Booking Class:\n", churn_counts)
print("\nAverage Profitability by Frequent Flyer Status:\n", avg_profit_ff)

# Save cleaned data for Power BI import
df.to_csv('cleaned_kaggle_airline_data.csv', index=False)


Churn counts by Booking Class:
 Booking Class
Business    13
Economy     25
First       15
Name: Churned, dtype: int64

Average Profitability by Frequent Flyer Status:
 Frequent Flyer Status
Gold        0.964066
Platinum    1.079723
Silver      1.060408
Name: Profitability, dtype: float64


In [9]:
# Check missing distances
print(f"Missing distances: {df['Distance_Miles'].isna().sum()}")

# Check Load Factor stats
print(df['Load_Factor'].describe())

# Sample route profitability
print(df.groupby('Route')['Profitability'].mean().sort_values(ascending=False).head(10))


Missing distances: 0
count    100.000000
mean       0.025000
std        0.018966
min        0.005556
25%        0.005556
50%        0.020000
75%        0.050000
max        0.050000
Name: Load_Factor, dtype: float64
Route
SYD-MEL    1.051237
BNE-SYD    1.027883
MEL-BNE    1.016267
Name: Profitability, dtype: float64


In [11]:
df.head()

,Departure City,Arrival City,Departure Date,Flight Duration,Delay Minutes,Customer ID,Name,Booking Class,Frequent Flyer Status,Route,...,Destination,Profitability,Loyalty Points,Churned,Distance_Miles,Estimated_Seats,Revenue,ASM,RPM,Load_Factor
0,Wilsonstad,Lake Johnmouth,2023-02-05 20:11:09,1.27581,120,3769,Daniel Oliver,Business,Gold,MEL-BNE,...,LHR,0.632226,4245,True,10503.522545,50,370.638128,5.251761e+05,10503.522545,0.020000
1,New Brent,Port Wanda,NaT,1.27581,35,3529,Deborah Hall,Economy,Platinum,BNE-SYD,...,SIN,1.265026,833,True,3743.172312,180,114.529016,6.737710e+05,3743.172312,0.005556
2,South Samanthaberg,Lake Meganside,2023-12-05 15:16:31,0.72111,67,1303,Mary York,Economy,Platinum,MEL-BNE,...,LAX,1.141651,2568,True,7921.180638,180,164.468018,1.425813e+06,7921.180638,0.005556
3,Lake Gracefurt,Jamesberg,NaT,-0.94299,72,2965,Christina Sanchez,Economy,Gold,BNE-SYD,...,LAX,1.129291,284,True,7921.180638,180,318.903167,1.425813e+06,7921.180638,0.005556
4,Owenborough,Kelleymouth,NaT,-0.38829,101,8779,Dustin Owens,Economy,Silver,BNE-SYD,...,SIN,1.218239,2805,True,3815.599428,180,389.971051,6.868079e+05,3815.599428,0.005556


In [12]:
df.to_csv('cleaned_airline_data.csv', index=False)